# Heatmap NDVI Wilayah Lembang
Visualisasi spasial indeks kehijauan vegetasi (NDVI) berdasarkan data rata-rata per titik dalam batas wilayah Lembang.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.path as mpath


## 1. Load Data

In [ ]:

df = pd.read_csv("Lembang_Merged_SPI_CLEANED.csv")
boundary_df = pd.read_csv("Koordinat_Polygon_Lembang.csv")


## 2. Hitung Rata-rata NDVI dan Filter Nilai Valid

In [ ]:

avg_ndvi = df.groupby(["lat", "lon"])["NDVI"].mean().reset_index()
avg_ndvi = avg_ndvi[(avg_ndvi["NDVI"] >= -1) & (avg_ndvi["NDVI"] <= 1)]


## 3. Interpolasi Grid dan Masking ke Batas Wilayah

In [ ]:

grid_lon = np.linspace(avg_ndvi["lon"].min(), avg_ndvi["lon"].max(), 300)
grid_lat = np.linspace(avg_ndvi["lat"].min(), avg_ndvi["lat"].max(), 300)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_z = griddata((avg_ndvi["lon"], avg_ndvi["lat"]), avg_ndvi["NDVI"], (grid_lon, grid_lat), method='linear')

boundary_coords = boundary_df[['lon', 'lat']].values
polygon_path = mpath.Path(boundary_coords)
points = np.vstack((grid_lon.flatten(), grid_lat.flatten())).T
mask = polygon_path.contains_points(points).reshape(grid_lon.shape)
masked_grid_z = np.where(mask, grid_z, np.nan)


## 4. Plot Heatmap NDVI

In [ ]:

plt.figure(figsize=(10, 10))
plt.contourf(grid_lon, grid_lat, masked_grid_z, levels=100, cmap="YlGn", alpha=0.9)
plt.colorbar(label="NDVI")
plt.contour(grid_lon, grid_lat, masked_grid_z, levels=15, linewidths=0.4, colors='black', alpha=0.3)
plt.plot(boundary_coords[:, 0], boundary_coords[:, 1], color="white", linewidth=1.5)
plt.grid(True, which='major', linestyle='--', linewidth=0.5, alpha=0.4)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap NDVI (Rata-rata Sepanjang Waktu) Wilayah Lembang")
plt.gca().set_facecolor("white")
plt.axis("equal")
plt.tight_layout()
plt.show()
